<a href="https://colab.research.google.com/github/justxoai/DeepLearning2024/blob/main/YOLOv8_seg_VS_YOLOv8_obb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LIBS

In [2]:
!pip install roboflow
from roboflow import Roboflow

!pip install -q ultralytics
from ultralytics import YOLO

import os
import shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.3/881.3 kB 20.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Delete auto-generated sample data by google colab

In [3]:
shutil.rmtree('sample_data', ignore_errors=True)

initialize get dataset version by project by workspace on roboflow

In [ ]:
%cd /content
rf = Roboflow(api_key="KRgH3Ot0uRDKLN7SDRqL")
project = rf.workspace("kfupm-7hvnm").project("lung-tumor-2xcgn")
version = project.version(3)

/content
loading Roboflow workspace...
loading Roboflow project...


refactoring folder names of datasets

In [ ]:
%cd /content
current_folder_name = 'Lung-Tumor-3'
new_folder_name = ['lung-tumor-seg', 'lung-tumor-obb']

dataset_seg = version.download("yolov8")
os.rename(current_folder_name, new_folder_name[0])
print(f"\nFolder renamed from {current_folder_name} to {new_folder_name[0]}")

dataset_obb = version.download("yolov8-obb")
os.rename(current_folder_name, new_folder_name[1])
print(f"\nFolder renamed from {current_folder_name} to {new_folder_name[1]}")

seg_data_path = os.path.join('/content/lung-tumor-seg', 'data.yaml')
obb_data_path = os.path.join('/content/lung-tumor-obb', 'data.yaml')

import yaml

with open(seg_data_path, 'r') as file:
    content = file.read()
content = content.replace(current_folder_name + '/', '')
with open(seg_data_path, 'w') as file:
    file.write(content)
with open(obb_data_path, 'r') as file:
    content = file.read()
content = content.replace('path: ../datasets/roboflow', '')
with open(obb_data_path, 'w') as file:
    file.write(content)


/content
Dependency ultralytics==8.0.196 is required but found version=8.2.103, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Lung-Tumor-3 in yolov8:: 100%|██████████| 698/698 [00:00<00:00, 2353.09it/s]



Folder renamed from Lung-Tumor-3 to lung-tumor-seg



Extracting Dataset Version Zip to Lung-Tumor-3 in yolov8-obb:: 100%|██████████| 698/698 [00:00<00:00, 3959.74it/s]



Folder renamed from Lung-Tumor-3 to lung-tumor-obb


In [ ]:
# reset (by deleting the dataset folders)
shutil.rmtree(new_folder_name[0])
shutil.rmtree(new_folder_name[1])

initializing YOLO model using pretrained weights provided by ultralytics

seg stands for segmentation

obb stands for oriented boundary box

In [ ]:
model_seg = YOLO("yolov8x-seg.pt")
model_obb = YOLO("yolov8x-obb.pt")

100%|██████████| 137M/137M [00:03<00:00, 39.1MB/s]


100%|██████████| 133M/133M [00:03<00:00, 41.0MB/s]


#### Training Model Parameters
In this topic, we are able to pass a number of arguments:

- **`imgsz:`** Define input image size  

    -> The size of the input image can significantly impact the model's accuracy and training speed. Larger images provide more detail but require more memory and computation time. Smaller images may speed up training but at the potential cost of losing important information.

- **`batch:`** Determine batch size  

    -> Batch size defines how many samples will be processed before the model's weights are updated. Larger batch sizes may result in more stable training but require more memory (especially on GPUs). Smaller batch sizes can reduce memory usage but might introduce noise in the weight updates, which could affect convergence.

- **`epochs:`** Define the number of training epochs

    -> An epoch refers to one complete pass through the entire training dataset. The more epochs, the more opportunities the model has to learn from the data. However, too many epochs can lead to overfitting.

- **`patience`**:
    -> The patience parameter controls how many epochs the training will continue after a metric (like validation loss) stops improving. It helps prevent overtraining and overfitting.

YOLOv8x-seg: SEGMENTATION

- **`weights:`** Specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint  
    
    -> In transfer learning, starting with a model that has already been trained on a large dataset allows your model to learn faster and potentially achieve better performance. Instead of starting from scratch, the model leverages knowledge learned from a related task.

- **`data:`** Our dataset location is saved in the `/content/lung-tumor-seg`

    -> This parameter points to where the training data is stored. It typically includes both images and their corresponding labels.

In [ ]:
%cd /content
seg_project = 'seg_project'
seg_name = '100_epochs'
seg_results = model_seg.train(
    data = seg_data_path,
    project = seg_project,
    name = seg_name,
    epochs = 100,
    patience = 10,
    batch = 4,
    imgsz = 640,
    workers = 8
)

/content
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=/content/lung-tumor-seg/data.yaml, epochs=100, time=None, patience=10, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=seg_project, name=100_epochs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

100%|██████████| 755k/755k [00:00<00:00, 67.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 392MB/s]


AMP: checks passed ✅


...

     27/100      5.05G        1.7      1.281     0.8697      1.208          7        640: 100%|██████████| 60/60 [00:24<00:00,  2.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.95it/s]

                   all         69         70      0.942      0.971      0.963      0.426      0.833      0.857      0.824      0.321
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 17, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



27 epochs completed in 0.264 hours.
Optimizer stripped from seg_project/100_epochs/weights/last.pt, 143.9MB
Optimizer stripped from seg_project/100_epochs/weights/best.pt, 143.9MB

Validating seg_project/100_epochs/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-seg summary (fused): 295 layers, 71,721,619 parameters, 0 gradients, 343.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:04<00:00,  2.10it/s]


                   all         69         70      0.939      0.871      0.937      0.426      0.955      0.886      0.951      0.396
Speed: 0.7ms preprocess, 34.2ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to seg_project/100_epochs


YOLOv8x-seg: SEGMENTATION

- **`weights:`** Specify a path to weights to start transfer learning from. Here we choose the generic DOTAv1 pretrained checkpoint  
    
    -> In transfer learning, starting with a model that has already been trained on a large dataset allows your model to learn faster and potentially achieve better performance. Instead of starting from scratch, the model leverages knowledge learned from a related task.

- **`data:`** Our dataset location is saved in the `/content/lung-tumor-seg`

    -> This parameter points to where the training data is stored. It typically includes both images and their corresponding labels.

In [ ]:
%cd /content
obb_project = 'obb_project'
obb_name = '100_epochs'
obb_results = model_obb.train(
    data = obb_data_path,
    project = obb_project,
    name = obb_name,
    epochs = 100,
    patience = 10,
    batch = 4,
    imgsz = 640,
    workers = 8
)

/content
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8x-obb.pt, data=/content/lung-tumor-obb/data.yaml, epochs=100, time=None, patience=10, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=obb_project, name=100_epochs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning /content/lung-tumor-obb/train/labels... 240 images, 0 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<00:00, 664.35it/s]

train: New cache created: /content/lung-tumor-obb/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/lung-tumor-obb/valid/labels... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<00:00, 468.19it/s]

val: New cache created: /content/lung-tumor-obb/valid/labels.cache


...

     19/100      6.26G      1.015     0.5903      1.433          6        640: 100%|██████████| 60/60 [00:21<00:00,  2.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.47it/s]

                   all         69         70      0.995      0.986      0.994      0.689
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



19 epochs completed in 0.241 hours.
Optimizer stripped from obb_project/100_epochs/weights/last.pt, 139.4MB
Optimizer stripped from obb_project/100_epochs/weights/best.pt, 139.4MB

Validating obb_project/100_epochs/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-obb summary (fused): 287 layers, 69,450,054 parameters, 0 gradients, 263.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.65it/s]


                   all         69         70          1      0.968      0.994       0.69
Speed: 0.6ms preprocess, 28.5ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to obb_project/100_epochs


In [ ]:
# reset
shutil.rmtree('seg_project')

In [ ]:
# reset
shutil.rmtree('obb_project')

THIS COPIES FILES FROM COLAB TO GOOGLE DRIVE

In [14]:
from google.colab import drive
drive.mount('/content/drive')

seg_dataset_dir = '/content/lung-tumor-seg'
obb_dataset_dir = '/content/lung-tumor-obb'
seg_results_dir = '/content/seg_project'
obb_results_dir = '/content/obb_project'
seg_results_noweights_dir = '/content/seg_project_noweights'
obb_results_noweights_dir = '/content/obb_project_noweights'

drive_dir = "/content/drive/MyDrive/DL"

shutil.make_archive(seg_dataset_dir, 'zip', seg_dataset_dir)
shutil.make_archive(obb_dataset_dir, 'zip', obb_dataset_dir)
shutil.make_archive(seg_results_dir, 'zip', seg_results_dir)
shutil.make_archive(obb_results_dir, 'zip', obb_results_dir)
shutil.make_archive(seg_results_noweights_dir, 'zip', seg_results_noweights_dir)
shutil.make_archive(obb_results_noweights_dir, 'zip', obb_results_noweights_dir)

shutil.move("/content/lung-tumor-seg.zip", os.path.join(drive_dir, "lung-tumor-seg.zip"))
shutil.move("/content/lung-tumor-obb.zip", os.path.join(drive_dir, "lung-tumor-obb.zip"))
shutil.move("/content/seg_project.zip", os.path.join(drive_dir, "seg_project.zip"))
shutil.move("/content/obb_project.zip", os.path.join(drive_dir, "obb_project.zip"))
shutil.move("/content/seg_project_noweights.zip", os.path.join(drive_dir, "seg_project_noweights.zip"))
shutil.move("/content/obb_project_noweights.zip", os.path.join(drive_dir, "obb_project_noweights.zip"))

print("Files moved successfully to Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files moved successfully to Google Drive.


ACTUAL

In [ ]:
import cv2
import numpy as np
import os

# Define paths
images_folder = "/content/lung-tumor-seg/test/images"
labels_folder = "/content/lung-tumor-seg/test/labels"  # Assuming masks are in the same folder as labels for YOLOv8
output_folder = "/content/lung-tumor-seg/test/actual"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through all image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check for image file extensions
        # Load the image
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)

        # Load the segmentation label
        label_path = os.path.join(labels_folder, os.path.splitext(image_file)[0] + '.txt')
        height, width, _ = image.shape

        # Check if label file exists
        if not os.path.exists(label_path):
            print(f"Label file {label_path} does not exist. Skipping...")
            continue

        with open(label_path, 'r') as file:
            labels = file.readlines()

        # Iterate through each object in the label
        for label in labels:
            label = label.strip().split()
            class_id = int(label[0])
            polygon_coords = list(map(float, label[1:]))

            # Reshape polygon coordinates
            polygon = np.array(polygon_coords).reshape(-1, 2)
            polygon[:, 0] = polygon[:, 0] * width  # Convert normalized x to pixel x
            polygon[:, 1] = polygon[:, 1] * height # Convert normalized y to pixel y
            polygon = polygon.astype(np.int32)

            # Draw the polygon mask on the image (for visualization)
            cv2.polylines(image, [polygon], isClosed=True, color=(0, 0, 255), thickness=2)
            cv2.fillPoly(image, [polygon], color=(255, 0, 255))  # Draw the filled mask

        # Save the modified image
        output_image_path = os.path.join(output_folder, image_file)
        save_success = cv2.imwrite(output_image_path, image)

        if save_success:
            print(f"Image saved successfully at {output_image_path}")
        else:
            print(f"Failed to save the image at {output_image_path}")


Image saved successfully at /content/lung-tumor-seg/test/actual/000021_02_01_068_png_jpg.rf.1a5ef4b2969421b2fb1e65728d0bf2b0.jpg
Image saved successfully at /content/lung-tumor-seg/test/actual/001404_01_01_139_png_jpg.rf.286b22d8ea74a86732ab9397d3553be4.jpg
Image saved successfully at /content/lung-tumor-seg/test/actual/001083_09_01_093_png_jpg.rf.8e1523e1e214bb04a5c296ced316db62.jpg
Image saved successfully at /content/lung-tumor-seg/test/actual/002265_02_01_031_png_jpg.rf.cac17609de187598bd537354b0ce3e75.jpg
Image saved successfully at /content/lung-tumor-seg/test/actual/000103_02_01_168_png_jpg.rf.76e5e9fffd849b95bf96ae1ed4b1be65.jpg
Image saved successfully at /content/lung-tumor-seg/test/actual/003380_03_01_140_png_jpg.rf.f79c35e472e03b850cd573ff0dd8b253.jpg
Image saved successfully at /content/lung-tumor-seg/test/actual/000230_05_01_165_png_jpg.rf.785131d9ef12d61057eadfd5e6b257ca.jpg
Image saved successfully at /content/lung-tumor-seg/test/actual/000188_03_01_146_png_jpg.rf.0aa9c

In [ ]:
import cv2
import numpy as np
import os

# Define paths
images_folder = "/content/lung-tumor-obb/test/images"
labels_folder = "/content/lung-tumor-obb/test/labels"  # Assuming masks are in the same folder as labels for YOLOv8
output_folder = "/content/lung-tumor-obb/test/actual"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through all image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check for image file extensions
        # Load the image
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)

        # Load the OBB label
        label_path = os.path.join(labels_folder, os.path.splitext(image_file)[0] + '.txt')
        height, width, _ = image.shape

        # Check if label file exists
        if not os.path.exists(label_path):
            print(f"Label file {label_path} does not exist. Skipping...")
            continue

        with open(label_path, 'r') as file:
            labels = file.readlines()

        # Iterate through each object in the label
        for label in labels:
            label = label.strip().split()
            class_id = int(label[0])  # Read class ID
            polygon_coords = list(map(float, label[1:]))  # Read polygon coordinates

            # Reshape polygon coordinates
            polygon = np.array(polygon_coords).reshape(-1, 2)
            polygon[:, 0] = polygon[:, 0] * width  # Convert normalized x to pixel x
            polygon[:, 1] = polygon[:, 1] * height # Convert normalized y to pixel y
            polygon = polygon.astype(np.int32)

            # Draw the polygon mask on the image (for visualization)
            cv2.polylines(image, [polygon], isClosed=True, color=(255, 0, 255), thickness=2)
            # cv2.fillPoly(image, [polygon], color=(0, 255, 0))  # Draw the filled mask

        # Save the modified image
        output_image_path = os.path.join(output_folder, image_file)
        save_success = cv2.imwrite(output_image_path, image)

        if save_success:
            print(f"Image saved successfully at {output_image_path}")
        else:
            print(f"Failed to save the image at {output_image_path}")


Image saved successfully at /content/lung-tumor-obb/test/actual/000021_02_01_068_png_jpg.rf.1a5ef4b2969421b2fb1e65728d0bf2b0.jpg
Image saved successfully at /content/lung-tumor-obb/test/actual/001404_01_01_139_png_jpg.rf.286b22d8ea74a86732ab9397d3553be4.jpg
Image saved successfully at /content/lung-tumor-obb/test/actual/001083_09_01_093_png_jpg.rf.8e1523e1e214bb04a5c296ced316db62.jpg
Image saved successfully at /content/lung-tumor-obb/test/actual/002265_02_01_031_png_jpg.rf.cac17609de187598bd537354b0ce3e75.jpg
Image saved successfully at /content/lung-tumor-obb/test/actual/000103_02_01_168_png_jpg.rf.76e5e9fffd849b95bf96ae1ed4b1be65.jpg
Image saved successfully at /content/lung-tumor-obb/test/actual/003380_03_01_140_png_jpg.rf.f79c35e472e03b850cd573ff0dd8b253.jpg
Image saved successfully at /content/lung-tumor-obb/test/actual/000230_05_01_165_png_jpg.rf.785131d9ef12d61057eadfd5e6b257ca.jpg
Image saved successfully at /content/lung-tumor-obb/test/actual/000188_03_01_146_png_jpg.rf.0aa9c

PREDICT

In [ ]:
import cv2
import os
from ultralytics import YOLO

# Define paths
images_folder = "/content/lung-tumor-seg/test/images"
output_folder = "/content/lung-tumor-seg/test/predict"

# Load your trained YOLOv8 segmentation model
model = YOLO("/content/seg_project/100_epochs/weights/best.pt")

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through all image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check for image file extensions
        # Load the image
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)

        # Predict with the model
        results = model(image_path)  # Predict on the image file

        # Iterate through the results
        for result in results:
            masks = result.masks  # Get masks from results
            names = model.names  # Get class names from the model

            # Draw the masks on the image
            if masks is not None and masks.data is not None:
                for i in range(masks.data.shape[0]):  # Iterate over each mask
                    mask_array = masks.data[i].cpu().numpy()  # Get mask data as NumPy array
                    mask_array = (mask_array > 0).astype(np.uint8)  # Binarize the mask

                    # Create a red color version of the mask
                    colored_mask = np.zeros_like(image)  # Create an empty image with the same shape
                    colored_mask[mask_array == 1] = [0, 255, 255]  # Set the mask region to red

                    # Blend the mask with the original image
                    alpha = 0.5  # Adjust transparency
                    blended = cv2.addWeighted(image, 1 - alpha, colored_mask, alpha, 0)

                    # Update the original image with the blended mask
                    image = blended

                    # Access confidence and class using boxes
                    if result.boxes is not None:
                        conf = result.boxes[i].conf.item()  # Confidence score
                        cls = result.boxes[i].cls.item()  # Class index
                        class_name = names[int(cls)]  # Get class name from index

                        # Calculate the position to draw the text (using mask's centroid)
                        y, x = np.where(mask_array == 1)  # Get coordinates of the mask
                        if len(x) > 0 and len(y) > 0:  # Check if there are any coordinates
                            text_x, text_y = int(x.mean()), int(y.mean())  # Get the centroid for text position
                            # Draw the class name and confidence next to the mask
                            cv2.putText(image, f'{class_name}: {conf:.2f}',
                                        (text_x, text_y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                                        1, (255, 255, 0), 2)  # White text

        # Save the modified image
        output_image_path = os.path.join(output_folder, image_file)
        save_success = cv2.imwrite(output_image_path, image)

        if save_success:
            print(f"Image saved successfully at {output_image_path}")
        else:
            print(f"Failed to save the image at {output_image_path}")



image 1/1 /content/lung-tumor-seg/test/images/000021_02_01_068_png_jpg.rf.1a5ef4b2969421b2fb1e65728d0bf2b0.jpg: 640x640 (no detections), 7995.8ms
Speed: 14.8ms preprocess, 7995.8ms inference, 12.8ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/lung-tumor-seg/test/predict/000021_02_01_068_png_jpg.rf.1a5ef4b2969421b2fb1e65728d0bf2b0.jpg

image 1/1 /content/lung-tumor-seg/test/images/001404_01_01_139_png_jpg.rf.286b22d8ea74a86732ab9397d3553be4.jpg: 640x640 1 Tumor, 5801.6ms
Speed: 2.5ms preprocess, 5801.6ms inference, 27.5ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/lung-tumor-seg/test/predict/001404_01_01_139_png_jpg.rf.286b22d8ea74a86732ab9397d3553be4.jpg

image 1/1 /content/lung-tumor-seg/test/images/001083_09_01_093_png_jpg.rf.8e1523e1e214bb04a5c296ced316db62.jpg: 640x640 1 Tumor, 6763.4ms
Speed: 2.7ms preprocess, 6763.4ms inference, 8.9ms postprocess per image at shape (1, 3, 640, 640)
Image saved suc

In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO

# Define paths
images_folder = "/content/lung-tumor-obb/test/images"
output_folder = "/content/lung-tumor-obb/test/predict"

# Load your trained YOLOv8 segmentation model
model = YOLO("/content/obb_project/100_epochs/weights/best.pt")

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Extract the class names from the model
names = model.names  # Get class names from the model

# Iterate through all image files in the images folder
for image_file in os.listdir(images_folder):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check for image file extensions
        # Load the image
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)

        # Predict with the model
        results = model(image_path)  # Predict on the image file

        # Iterate through the results
        for result in results:
            # Check if the result contains OBB
            if hasattr(result, 'obb') and result.obb is not None:
                # Get OBB points
                points = result.obb.xyxyxyxy.cpu().numpy()  # Retrieve OBB points (8-point format)
                conf = result.obb.conf.cpu().numpy()  # Retrieve confidence scores
                cls = result.obb.cls.cpu().numpy()  # Retrieve class labels

                for i in range(len(points)):
                    # Convert points to integer for drawing
                    point_coords = points[i].reshape(4, 2).astype(np.int32)  # Reshape to get 4 points (x, y)

                    # Draw OBB points as lines
                    cv2.polylines(image, [point_coords], isClosed=True, color=(0, 255, 255), thickness=2)  # Green for OBB

                    # Draw the confidence and class label
                    cv2.putText(image, f'{names[int(cls[i])]}: {conf[i]:.2f}',
                                (point_coords[0][0], point_coords[0][1]-10), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (255, 255, 0), 2)

        # Save the modified image
        output_image_path = os.path.join(output_folder, image_file)
        save_success = cv2.imwrite(output_image_path, image)

        if save_success:
            print(f"Image saved successfully at {output_image_path}")
        else:
            print(f"Failed to save the image at {output_image_path}")



image 1/1 /content/lung-tumor-obb/test/images/000021_02_01_068_png_jpg.rf.1a5ef4b2969421b2fb1e65728d0bf2b0.jpg: 640x640 4749.5ms
Speed: 2.6ms preprocess, 4749.5ms inference, 7.6ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/lung-tumor-obb/test/predict/000021_02_01_068_png_jpg.rf.1a5ef4b2969421b2fb1e65728d0bf2b0.jpg

image 1/1 /content/lung-tumor-obb/test/images/001404_01_01_139_png_jpg.rf.286b22d8ea74a86732ab9397d3553be4.jpg: 640x640 4579.4ms
Speed: 3.3ms preprocess, 4579.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/lung-tumor-obb/test/predict/001404_01_01_139_png_jpg.rf.286b22d8ea74a86732ab9397d3553be4.jpg

image 1/1 /content/lung-tumor-obb/test/images/001083_09_01_093_png_jpg.rf.8e1523e1e214bb04a5c296ced316db62.jpg: 640x640 6333.9ms
Speed: 2.4ms preprocess, 6333.9ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/lung-tumor-obb/t

GET FILE FROM DRIVE

In [5]:
shutil.copy('/content/drive/MyDrive/DL/lung-tumor-seg.zip', '/content/')
shutil.copy('/content/drive/MyDrive/DL/lung-tumor-obb.zip', '/content/')
shutil.copy('/content/drive/MyDrive/DL/seg_project.zip', '/content/')
shutil.copy('/content/drive/MyDrive/DL/obb_project.zip', '/content/')

'/content/obb_project.zip'

UNZIP FILES

In [6]:
shutil.unpack_archive('lung-tumor-seg.zip', '/content/lung-tumor-seg/')
shutil.unpack_archive('lung-tumor-obb.zip', '/content/lung-tumor-obb/')
shutil.unpack_archive('seg_project.zip', '/content/seg_project/')
shutil.unpack_archive('obb_project.zip', '/content/obb_project/')

In [7]:
os.makedirs('/content/seg_project_noweights/')
os.makedirs('/content/obb_project_noweights/')


REPLICATE CONTENT

In [11]:
import os
import shutil

def copy_folder_contents(src_folder, dest_folder):
    # Check if the source folder exists
    if not os.path.exists(src_folder):
        print(f"Source folder '{src_folder}' does not exist.")
        return

    # Create destination folder if it doesn't exist
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # Iterate through all the files and directories in the source folder
    for item in os.listdir(src_folder):
        src_path = os.path.join(src_folder, item)
        dest_path = os.path.join(dest_folder, item)

        # Copy files and directories
        if os.path.isdir(src_path):
            shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
        else:
            shutil.copy2(src_path, dest_path)

    print(f"All contents copied from '{src_folder}' to '{dest_folder}'.")

# Example usage
src_folder = '/content/obb_project/'
dest_folder = '/content/obb_project_noweights/'
copy_folder_contents(src_folder, dest_folder)


All contents copied from '/content/obb_project/' to '/content/obb_project_noweights/'.


In [13]:
shutil.rmtree('/content/seg_project_noweights/100_epochs/weights')
shutil.rmtree('/content/obb_project_noweights/100_epochs/weights')